# 读取聚宽原始数据

In [1]:
today="2023-11-03"

In [2]:
import pandas as pd
import numpy as np
Close=pd.read_csv("../期货数据/"+today+"/Afuturesclose1m.csv",index_col=0)
Open=pd.read_csv("../期货数据/"+today+"/Afuturesopen1m.csv",index_col=0)
High=pd.read_csv("../期货数据/"+today+"/Afutureshigh1m.csv",index_col=0)
Low=pd.read_csv("../期货数据/"+today+"/Afutureslow1m.csv",index_col=0)
Volume=pd.read_csv("../期货数据/"+today+"/Afuturesvolume1m.csv",index_col=0)
Money=pd.read_csv("../期货数据/"+today+"/Afuturesmoney1m.csv",index_col=0)

In [3]:
Open=Open.dropna(axis=1,how='all')
High=High.dropna(axis=1,how='all')
Low=Low.dropna(axis=1,how='all')
Close=Close.dropna(axis=1,how='all')
Volume=Volume.dropna(axis=1,how='all')
Money=Money.dropna(axis=1,how='all')

# 把每天的数据数量修改到555以便于切片

In [4]:
#将每天的数据数量修改到555
def fix_df_index_to_555(Close):
    date_list=[]
    time_list=[]
    for d in Close.index:
        if d.split(" ")[0] not in date_list:
            date_list.append(d.split(" ")[0])
        if d.split(" ")[1] not in time_list:
            time_list.append(d.split(" ")[1])
    print(len(date_list))
    print(len(time_list))

    d_and_t=[]
    for date in date_list:
        for time in time_list:
            d_and_t.append(date+" "+time)
    
    df=pd.DataFrame(index=d_and_t,columns=Close.columns)
    for d in Close.index:
        for futures in Close.columns:
            df.loc[d,futures]=Close.loc[d,futures]
    return df

In [5]:
Open=fix_df_index_to_555(Open)
High=fix_df_index_to_555(High)
Low=fix_df_index_to_555(Low)
Close=fix_df_index_to_555(Close)
Volume=fix_df_index_to_555(Volume)
Money=fix_df_index_to_555(Money)

863
555
863
555
863
555
863
555
863
555
863
555


In [6]:
Open.to_csv("fix_555_Open.csv")
High.to_csv("fix_555_High.csv")
Low.to_csv("fix_555_Low.csv")
Close.to_csv("fix_555_Close.csv")
Money.to_csv("fix_555_Money.csv")
Volume.to_csv("fix_555_Volume.csv")

# 再进行na的填充，用最新一条非na的值填充所有的na

In [7]:
from tqdm import trange


def fix_na_in_df(df):#如果a行b列的值为na,用b列里离a行最近的非na值代替na，要求该非na的值的索引在a行之前
    #for d in trange(len(df.index)):#电脑卡死
    for d in range(len(df.index)):
        for futures in df.columns:
            today_index=Close.index[d]
            yesterday_index=Close.index[d-1]
            if str(df.loc[today_index,futures])=='nan' and yesterday_index!= df.index[-1] and str(df.loc[yesterday_index,futures])!='nan':
                df.loc[today_index,futures]=df.loc[yesterday_index,futures]
        
    return df


In [8]:
from tqdm import trange


def fix_na_in_money_and_volume(df):#如果a行b列的值为na,用b列里离a行最近的非na值代替na，要求该非na的值的索引在a行之前
    #for d in trange(len(df.index)):#电脑卡死
    for d in range(len(df.index)):
        for futures in df.columns:
            today_index=Close.index[d]
            yesterday_index=Close.index[d-1]
            if str(df.loc[today_index,futures])=='nan' and yesterday_index!= df.index[-1] and str(df.loc[yesterday_index,futures])!='nan':
                df.loc[today_index,futures]=0
        
    return df


In [9]:
Open=fix_na_in_df(Open)
High=fix_na_in_df(High)
Low=fix_na_in_df(Low)
Close=fix_na_in_df(Close)
Volume=fix_na_in_money_and_volume(Volume)
Money=fix_na_in_money_and_volume(Money)

# 写入硬盘备用

In [10]:
Open.to_csv("Open.csv")
High.to_csv("High.csv")
Low.to_csv("Low.csv")
Close.to_csv("Close.csv")
Money.to_csv("Money.csv")
Volume.to_csv("Volume.csv")